In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/sample/perceptron_sampledata.csv


In [2]:
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.nn.functional as f

In [3]:
df=pd.read_csv("/kaggle/input/sample/perceptron_sampledata.csv")
df.head()

,x1,x2,label
0,0.77,-1.14,0
1,-0.33,1.44,0
2,0.91,-3.07,0
3,-0.37,-1.91,0
4,-0.63,-1.53,0


In [4]:
y = df['label'].values
x = df.drop(columns=['label'],axis=1).values

In [5]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(x,y,test_size=0.2,random_state = 0)

In [6]:
X_train = torch.FloatTensor(X_train)
X_test = torch.FloatTensor(X_test)
y_train = torch.FloatTensor(y_train)
y_test = torch.FloatTensor(y_test)

In [7]:
class ANN_Model(nn.Module):
    def __init__(self,input_features=2,hidden1=1,hidden2=2,out_features=1):
        super().__init__()
        self.layer1 = nn.Linear(input_features,hidden1)
        self.layer2 = nn.Linear(hidden1,hidden2)
        self.out = nn.Linear(hidden2,out_features)
    
    def forward(self,x):
        x=f.tanh(self.layer1(x))
        x=f.tanh(self.layer2(x))
        x =self.out(x)
        return x

In [8]:
torch.manual_seed(3)
model = ANN_Model().to('cpu')
model

ANN_Model(
  (layer1): Linear(in_features=2, out_features=1, bias=True)
  (layer2): Linear(in_features=1, out_features=2, bias=True)
  (out): Linear(in_features=2, out_features=1, bias=True)
)

In [9]:
model.parameters()

<generator object Module.parameters at 0x7ebcaf4dda10>

In [10]:
#Backward Propagation
loss_function = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(params = model.parameters(),lr=0.01)

In [11]:
epochs = 200
final_losses = []
for i in range(epochs):
    model.train()
    y_pred = model(X_train).squeeze()
    loss = loss_function(y_pred,y_train)
    final_losses.append(loss.item())
    print("Epoch:{} and the loss: {}".format(i,loss.item()))
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

Epoch:0 and the loss: 20.97280502319336
Epoch:1 and the loss: 20.84825325012207
Epoch:2 and the loss: 20.720531463623047
Epoch:3 and the loss: 20.589962005615234
Epoch:4 and the loss: 20.45701026916504
Epoch:5 and the loss: 20.32225799560547
Epoch:6 and the loss: 20.18640899658203
Epoch:7 and the loss: 20.050228118896484
Epoch:8 and the loss: 19.914527893066406
Epoch:9 and the loss: 19.780128479003906
Epoch:10 and the loss: 19.647815704345703
Epoch:11 and the loss: 19.518314361572266
Epoch:12 and the loss: 19.39226531982422
Epoch:13 and the loss: 19.270214080810547
Epoch:14 and the loss: 19.152591705322266
Epoch:15 and the loss: 19.039714813232422
Epoch:16 and the loss: 18.931804656982422
Epoch:17 and the loss: 18.828968048095703
Epoch:18 and the loss: 18.73124122619629
Epoch:19 and the loss: 18.63858413696289
Epoch:20 and the loss: 18.55089569091797
Epoch:21 and the loss: 18.46803855895996
Epoch:22 and the loss: 18.389833450317383
Epoch:23 and the loss: 18.316082000732422
Epoch:24 and

In [12]:
def accuracy_fn(y_true, y_pred):
    correct = torch.eq(y_true, y_pred).sum().item()
    acc = (correct/len(y_pred))*100
    return acc

In [13]:
model.eval()
with torch.inference_mode():
    # forward pass
    test_logits = model(X_test).squeeze()
    test_pred = torch.round(torch.sigmoid(test_logits))
    test_loss = loss_function(test_logits,y_test)
    test_cc = accuracy_fn(y_true=y_test,y_pred=test_pred)
print(test_cc)

75.0
